# Complete code for 3T - 7T analysis

Steps:    
0. DEMOGRAPHICS  
    - Identify IDs with 3T and 7T    
    - Extract clinical information for epilepsy patients  
    - Extract demographic information for all participants  
1. SMOOTH MAPS
2. CLEAN DATA
3. ANALYSES
    - within study TLE vs CTRL comparison
        - extract smoother maps
        - compute z, w scores (values per participant)
        - group and flip
        - Cohen's D (compare TLE and control z/w score distributions within each vertex)
    - between study 7T vs 3T comparison      
4. Figures  


# 0. DEMOGRAPHICS

In [59]:
import os
import pandas as pd
import sys
import importlib
import re
import numpy as np
import datetime

sys.path.append(os.getcwd())
#sys.path.append("/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/code")
sys.path.append("/host/verges/tank/data/daniel/")
import demo
import tTsTGrpUtils as tsutil
from genUtils import id, gen, t1

importlib.reload(demo)
importlib.reload(id)
importlib.reload(gen)
importlib.reload(t1)
importlib.reload(tsutil)


<module 'tTsTGrpUtils' from '/host/verges/tank/data/daniel/3T7T/z/code/analyses/tTsTGrpUtils.py'>

In [ ]:
lab = True
save = True
verbose = True
toPrint = True

if lab: # define root paths to source files
    src_dir = "/host/verges/tank/data/daniel/3T7T/z/data/sources" # path to directory with source pt sheets
    save_pth = save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs"
else:
    src_dir = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/PT/sources" # path to directory with source pt sheets
    save_pth = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/data/outputs"


# For each sheet, must define NAME, PATH, SHEET, ID_7T, ID_3T. 
# All other keys are those to be extracted.
# The same variables should have the same key names across sheets.
PNI = {
    'NAME': 'PNI',
    'PATH': f'{src_dir}/MICA_PNI_27Aug2025.xlsx', # 7T controls
    'SHEET': 'all', # name of sheet in file
    'ID_7T': 'ID_PNI', 
    'ID_3T': 'ID_MICs',
    'Ses_7T': 'session',
    'Date_7T': 'scanDate',
    'study': '7T',
    'DOB': 'dob',
    'Sex': 'sex',
    'Gender': 'gender',
    'Hand': 'handedness',
    'Eth': 'ethnicity',
    'Language': 'language',
    'Job': 'employment',
    'Edu': 'education',
    'LastSz': 'lastSeizure'
}

MICs = {
    'NAME': 'MICs',
    'PATH': f'{src_dir}/MICA-MTL-3T_27Aug2025.xlsx', # 3T controls
    'SHEET': 'Sheet1', # name of sheet in file
    'ID_7T': None, 
    'ID_3T': 'Study_name',
    'Ses_3T': 'Visit',
    'Date_3T': 'Scan_Date (D.M.Y)',
    'study': '3T',
    'Hand': 'Handed', 
    'Sex': 'AssignedSex',
    'Gender': 'GenderIdentity',
    'Height': 'HeightApprox',
    'Weight': 'WeightApprox',
    'Eth': 'Ethnicity',
    'Job': 'Employ',
    'Edu': 'YoE',
    'LastSz': 'Last seizure'
}

Clin = {
    'NAME': 'Clin',
    'PATH': f'{src_dir}/Clinical_27Aug2025.xlsx',
    'SHEET': 'clinical-database-detailed', # name of sheet in file
    'ID_7T': None, 
    'ID_3T': 'participant_id',
    'Date_3T': None,
    'Gender': 'Gender',
    'Hand': 'Handedness',
    'Language': 'Language',
    'Job': 'Employment',
    'Edu': 'Education',
    'EpilepsyDxILAE': 'Epilepsy diagnosis based on ILAE',
    'EpilepsyClass': 'Epilepsy classification:Focal,Generalized',
    'FocusLat': 'Lateralization of epileptogenic focus',
    'FocusConfirmed': 'Epileptogenic focus confirmed by the information of (sEEG/ site of surgical resection/ Ictal EEG abnormalities +/. MRI findings): FLE=forntal lobe epilepsy and cingulate epilepsy, CLE:central/midline epilepsy,ILE: insular epilepsy, mTLE=mesio.temporal lobe epilepsy, nTLE=neocortical lobe epilepsy, PQLE=posterior quadrant lobe epilepsy , multifocal epilepsy,IGE=ideopathic lobe epilepsy,unclear)',
    'EMUDischargeDx': 'Dx at EMU discharge ',
    'EMUAdmissionDate': 'EMU admission date(dd-mm-yy)',
    'AdmissionDuration': 'Duration of admission',
    'EpilepsyRiskFactors': 'Risk factors for epilepsy',
    'SeizureOnsetYr': 'Seizure onset (yr)',
    'DrugResistant': 'Drug resistant epilepsy at time of EMU admission',
    'NumASMsPrior': '# of ASMs prior current EMU admission',
    'PrevASMs': 'Previous ASMs (name and doses (mg/d)) if applicable prior the current EMU admission',
    'NumASMOnAdmission': '# of ASM on admission',
    'ASMsOnAdmission': 'ASMs  on admission (name, doses (mg per day)',
    'GeneticTest': 'Genetic test (year,results)',
    'FDGPET': 'FDG.PET',
    'BaselineMRI': 'Baseline MRI (year,results)',
    'InvasiveExplorations': 'Invasive explorations (Y/N)',
    'NumSurgicalResections': '# of surgical resection/thermocoagulatin',
    'SurgicalResectionDateSite': 'Surgical resection date and site',
    'Histopathology': 'Histopatholgy',
    'Engel6mo': 'Engel classification (seizure outcomes at the 6 month )',
    'Engel1yr': 'Engel classification (seizure outcomes after 1 year from surgical resection)',
    'ILAEOutcome1yr': 'ILAE outcome after surgical resection by 1 yr',
    'NeuromodDevices': 'Neuromodulation devices'
    }

sheets = [PNI, MICs, Clin]
correspSheets = [PNI]

In [47]:
importlib.reload(demo)
df_demo, demo_csv_pth = demo.get_demo(sheets, save_pth=save_pth)

[ID_3T7T] Extracting IDs for participants with both 3T and 7T IDs
	Loading: PNI
	Skipping: MICs (missing ID_7T or ID_3T key)
	Skipping: Clin (missing ID_7T or ID_3T key)
[id_visits] Extracting visits for IDs in list
	Skipping: Clin (missing Ses_7T and Ses_3T key)


	PNI
		Extracting cols: ['ID_MICs', 'ID_PNI', 'scanDate', 'session']
	MICs
		Extracting cols: ['Study_name', 'Scan_Date (D.M.Y)', 'Visit']
		Finding corresponding ID_7T
[get_demo] There are  63   unique participants and  203  rows in datasheet.
[demo] Retrieving demographics data.
	Overlapping: ['Sex', 'Gender', 'Hand', 'Eth', 'Language', 'Job', 'Edu', 'LastSz', 'Date_3T']
	PNI
		merge_keys: ['ID_7T', 'ID_3T', 'Ses_7T']
	MICs
		merge_keys: ['ID_3T', 'Ses_3T']
	Clin
		merge_keys: ['ID_3T']
	Removing empty columns: ['WeightApprox', 'Employ', 'GenderIdentity', 'Ethnicity', 'Handed', 'AssignedSex', 'YoE', 'Scan_Date (D.M.Y)', 'HeightApprox'] 
[rmvNADate] No rows with missing scan dates found.
Index(['MICS_ID', 'PNI_ID', 'study',

In [30]:
df_demo

,UID,MICS_ID,PNI_ID,study,SES,Date,language,handedness,employment,dob,...,Surgical resection date and site,Seizure onset (yr),# of ASM on admission,Histopatholgy,Engel classification (seizure outcomes after 1 year from surgical resection),Drug resistant epilepsy at time of EMU admission,Dx at EMU discharge,age,grp,grp_detailed
0,UID0001,HC129,Pilot013,7T,05,18.04.2024,en,L,Full time student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
1,UID0001,HC129,Pilot013,3T,01,09.07.2024,en,L,MSc student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
2,UID0002,HC082,PNC003,7T,01,06.05.2022,English,R,Full time student,17.09.1997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.632444,CTRL,CTRL
3,UID0002,HC082,PNC003,7T,03,13.03.2023,English,R,Full time student,17.09.1997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.483915,CTRL,CTRL
4,UID0002,HC082,PNC003,7T,02,13.06.2022,English,R,Full time student,17.09.1997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.736482,CTRL,CTRL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,UID0061,PX064,PNE041,3T,03,21.01.2025,fr,R,Call center/bank,01.10.1997,...,NaN,13,1,NaN,NaN,N,JME,27.307324,"PATTERN NOT RECOGNIZED: lobe=IGE, lat=IGE, MFC...","PATTERN NOT RECOGNIZED: lobe=IGE, lat=IGE, MFC..."
196,UID0062,PX192,PNE042,7T,a1,26.08.2025,English,R,part-time freelance,04.06.1964,...,NaN,20,2,NaN,NaN,Y,left TLE as per clinical note,61.226557,TLE,TLE_L
197,UID0062,PX192,PNE042,3T,01,20.11.2024,en,R,part-time freelance,04.06.1964,...,NaN,20,2,NaN,NaN,Y,left TLE as per clinical note,60.462697,TLE,TLE_L
198,UID0063,PX231,PNE043,7T,a1,27.08.2025,French,R,Student cegep,21.08.2001,...,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,Followed at the CHUM,R TLE-HS,24.016427,TLE,TLE_R


In [43]:
# Eventually can be replaced/improved by a print Table 1 function
importlib.reload(demo)
demo.grp_summary(df_demo, col_grp='grp_detailed', save_pth=save_pth)
print("-"*100)
print("MEDIAN AGE by group")
df_demo.groupby(['grp_detailed', 'study'])['age'].median().sort_index(level='grp_detailed')

[grp_summary] Saved participant summary to /host/verges/tank/data/daniel/3T7T/z/outputs/00c_grpSummary_12Sep2025.csv
----------------------------------------
                                                                                         num_px  num_ses_3T  num_ses_7T max_ses_3T max_ses_7T median_ses_3T median_ses_7T
TOTAL                                                                                        63          94         106                                                  
CTRL                                                                                         20          38          58          4          6           2.0           3.0
TLE_R                                                                                         9          14           9          4          1           1.0           1.0
TLE_L                                                                                         7           8           8          2          2           1.0       

grp_detailed                                                                             study
CTRL                                                                                     3T       28.544832
                                                                                         7T       27.408624
FLE_L                                                                                    3T       24.906229
                                                                                         7T       25.407255
FLE_R                                                                                    3T       33.519507
                                                                                         7T       34.212183
MFCL                                                                                     3T       34.135524
                                                                                         7T       34.214921
PATTERN NOT RECOGNIZED: lobe=IGE, lat=IGE

# ANALYSIS SPECS

In [ ]:
# below parameters may eventually be moved to a config file

# run parameters
verbose = True
save = True
test = False
test_frac = 0.01 # fraction of demo to use for testing if test=True

# details of demographics file
demographics = {
    "pth" : demo_csv_pth,
    # column names:
    'nStudies': True, # whether multiple studies are included
    "ID_7T" : "PNI_ID", 
    "ID_3T" : "MICS_ID",
    "SES" : "SES",
    "date": "Date",
    "age": "age",
    "sex": "sex",
    "grp" : "grp_detailed" # col name for participant grouping variable to use
}

# specify root directories
MICs = {
    "name": "MICs",
    "dir_root": "/data/mica3/BIDS_MICs",
    "dir_raw": "/rawdata",
    "dir_deriv": "/derivatives",
    "dir_mp": "/micapipe_v0.2.0",
    "dir_hu": "/hippunfold_v1.3.0/hippunfold",
    "dir_zb": "/DM_zb_37comp",
    "study": "3T",
    "ID_ctrl" : ["HC"], # patterns for control IDs in demographics file
    "ID_Pt" : ["PX"] # patterns for patient IDs in demographics file
    }

PNI = {
    "name": "PNI",
    "dir_root": "/data/mica3/BIDS_PNI",
    "dir_raw": "/rawdata",
    "dir_deriv": "/derivatives",
    "dir_mp": "/micapipe_v0.2.0",
    "dir_hu": "/hippunfold_v1.3.0/hippunfold",
    "dir_zb": "/DM_zb_37comp",
    "study": "7T",
    "ID_col" : ["PNC", "Pilot"], # column for ID in demographics file
    }

studies = [MICs, PNI]

ctrl_grp = {'ctrl' : ['CTRL']}

px_grps = { # specify patient group labels to compare to controls
    'allPX' : ['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'],
    'TLE' : ['TLE_L', 'TLE_R', 'TLE_U', 'mTLE_R', 'mTLE_L'],
    'TLE_L': ['TLE_L', 'mTLE_L', 'bTLE_L'],
    'TLE_R': ['TLE_R', 'mTLE_R', 'bTLE_R'],
    'FCD' : ['FLE_R', 'FLE_L'],
    'MFCL' : ['MFCL', 'bTLE'],
    'UKN' : ['UKN_L', 'UKN_U']
}

# Make list of dict items for group definitions
groups = [
    {'TLE_L': px_grps['TLE_L']},
    {'TLE_R': px_grps['TLE_R']},
    ctrl_grp
]

specs  = { # all spec values to be in lists to allow for iteration across these values
    'prjDir_root' : "/host/verges/tank/data/daniel/3T7T/z", # output directory for smoothed cortical maps
    'prjDir_outs' : "/outputs",
    'prjDir_out_stats': "/outputs/stats",
    'prjDir_out_figs': "/outputs/figures",
    'prjDir_maps' : "/maps",
    'prjDir_dictLists': "/maps/dictLists",
    'prjDir_mapPths' : "/output/paths",

    'ctx': True, # whether to include cortical analyses
    'surf_ctx': ['fsLR-5k'],
    'lbl_ctx': ['midthickness'], # pial, midthick, white, etc
    'ft_ctx': ['thickness', 'T1map', 'flair'], # features: T1map, flair, thickness, FA, ADC
    'smth_ctx': [5, 10], # in mm
    
    'hipp': True, # whether to include hippocampal analyses
    'surf_hipp': ['0p5mm'],
    'lbl_hipp': ['midthickness'], # outter, inner, midthickness, etc
    'ft_hipp': ['thickness', 'T1map', 'flair'], # features: T1map, flair, thickness, FA, ADC
    'smth_hipp': [2, 5] # in mm
}


out_pths = {
    'df_00b_demo_pth' = demographics['pth']
} # to hold paths to generated outputs

In [53]:
# ensure demo file loaded
if df_demo is None:
    df_demo = pd.read_csv(demographics['pth'])
    print(f"[main] Demo file loaded from {demographics['pth']}")
    
if test:
    # take a random 10% subset of demo for testing
    df_demo = df_demo.sample(frac=test_frac).reset_index(drop=True)
    df_demo = df_demo.dropna(axis=1, how='all') # drop empty columns
    print(f"[TEST MODE] Running on random 10% subset of demographics ({len(df_demo)} rows).")

print(df_demo[['UID','MICS_ID', 'PNI_ID', 'study', 'SES', 'Date', 'grp_detailed']])

         UID MICS_ID    PNI_ID study SES        Date                                                 grp_detailed
0    UID0001   HC129  Pilot013    7T  05  18.04.2024                                                         CTRL
1    UID0001   HC129  Pilot013    3T  01  09.07.2024                                                         CTRL
2    UID0002   HC082    PNC003    7T  01  06.05.2022                                                         CTRL
3    UID0002   HC082    PNC003    7T  03  13.03.2023                                                         CTRL
4    UID0002   HC082    PNC003    7T  02  13.06.2022                                                         CTRL
..       ...     ...       ...   ...  ..         ...                                                          ...
195  UID0061   PX064    PNE041    3T  03  21.01.2025  PATTERN NOT RECOGNIZED: lobe=IGE, lat=IGE, MFCL=Generalized
196  UID0062   PX192    PNE042    7T  a1  26.08.2025                                    

# 1. SMOOTH MAPS
Strategy:
Add paths to relevant maps to df containing demographic information. Each row is one participant at a unique session.

Hippocampal maps: identify path to smoothed hippocampal maps, add to row-wise df
Cortical maps: take raw maps from micapipe, apply smoothing then save these maps in project directory and add path of the smoothed map to the df

In [ ]:
importlib.reload(tsutil)
df_pths, log = tsutil.idToMap(df_demo, studies, demographics, specs, verbose=True)

# Save DataFrames with map paths
if test:
    out_pth = f"{specs['prjDir_root']}{specs['prjDir_outs']}/01a_TEST_mapPths_{datetime.datetime.now().strftime('%d%b%Y')}.csv"
    log_pth = f"{specs['prjDir_root']}{specs['prjDir_outs']}/01a_TEST_mapPthsLOG_{datetime.datetime.now().strftime('%d%b%Y')}.log"
else:
    out_pth = f"{specs['prjDir_root']}{specs['prjDir_outs']}/01a_TEST_mapPths_{datetime.datetime.now().strftime('%d%b%Y-%H%M%S')}.csv"
    log_pth = f"{specs['prjDir_root']}{specs['prjDir_outs']}/01a_TEST_mapPthsLOG_{datetime.datetime.now().strftime('%d%b%Y-%H%M%S')}.log"

df_pths.to_csv(out_pth, index=False)
print(f"\n\n[main] Saved df with map paths: {out_pth}")

# Write the log string to file, preserving line breaks and tabs
with open(log_pth, 'w') as f:
    f.write(log)
print(f"[main] Saved mapPthsLOG log: {log_pth}")
out_pths['df_01a_mapPths'] = out_pth


[idToMap] idToMap function start time: 12-Sep-2025 17:01:42.
	 Finding/computing smoothed maps for provided surface, label, feature and smoothing combinations. Adding paths to dataframe...
[idToMap] 0 of 199...

7T sub-Pilot013 ses-05

	CORTICAL MAPS [7T sub-Pilot013 ses-05]...
	thickness, midthickness, fsLR-5k, smth-5mm
		Smoothed maps exists, adding to df: /host/verges/tank/data/daniel/3T7T/z/maps/sub-Pilot013_ses-05/sub-Pilot013_ses-05_ctx_hemi-L_surf-fsLR-5k_label-thickness_smth-5mm.func.gii	/host/verges/tank/data/daniel/3T7T/z/maps/sub-Pilot013_ses-05/sub-Pilot013_ses-05_ctx_hemi-R_surf-fsLR-5k_label-thickness_smth-5mm.func.gii

	thickness, midthickness, fsLR-5k, smth-10mm
		Smoothed maps exists, adding to df: /host/verges/tank/data/daniel/3T7T/z/maps/sub-Pilot013_ses-05/sub-Pilot013_ses-05_ctx_hemi-L_surf-fsLR-5k_label-thickness_smth-10mm.func.gii	/host/verges/tank/data/daniel/3T7T/z/maps/sub-Pilot013_ses-05/sub-Pilot013_ses-05_ctx_hemi-R_surf-fsLR-5k_label-thickness_smth-10mm.fu

OSError: Cannot save file into a non-existent directory: '/host/verges/tank/data/daniel/3T7T/z/output/paths'